
# 本番運用の品質モニタリング (スコアラーの自動実行)

https://docs.databricks.com/aws/ja/mlflow3/genai/eval-monitor/run-scorer-in-prod

In [0]:
%pip install -U "mlflow[databricks]>=3.1.1" openai

%restart_python

INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/24.7 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 24.6/24.7 MB 165.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 108.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.9 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/755.0 kB ? eta -:--:--
   ━━

## Traceデータの作成

In [0]:
import mlflow
from openai import OpenAI
from mlflow.entities import Document
from typing import List, Dict

# OpenAI呼び出しの自動トレースを有効にする
mlflow.openai.autolog()

# 同じ資格情報を使用してOpenAI経由でDatabricks LLMに接続する
# あるいは、ここで独自のOpenAI資格情報を使用することもできます
mlflow_creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
client = OpenAI(
    api_key=mlflow_creds.token, base_url=f"{mlflow_creds.host}/serving-endpoints"
)


@mlflow.trace
def generate_feature_explanation(system_name: str, feature_name: str) -> Dict[str, str]:
    """指定されたシステム/モジュール名と機能名について、説明を作成する"""

    # 取得したコンテキストを使用してメールを生成する
    prompt = f"""あなたは機械学習やデータエンジニアリングのエキスパートです。
    下記のシステム名および機能名で示される機能について、解説してください。

    システム名:
    {system_name}

    機能名:
    {feature_name}
    
    説明は簡潔かつ箇条書きでまとめてください。"""

    response = client.chat.completions.create(
        model="databricks-meta-llama-3-3-70b-instruct",
        messages=[
            {"role": "system", "content": "あなたは役に立つアシスタントです。"},
            {"role": "user", "content": prompt},
        ],
        max_tokens=2000,
    )

    return {"explanation": response.choices[0].message.content}


# トレースを作成する
generate_feature_explanation("MLflow", "ChatAgent")
generate_feature_explanation("MLflow", "Tracing")
generate_feature_explanation("Databricks", "Unity Catalog")
generate_feature_explanation("Databricks", "Agent Evaluation")
generate_feature_explanation("PowerBI", "Dashboard")

{'explanation': 'PowerBIのダッシュボード機能については以下の点が重要です。\n\n*   レポートやデータセットなどの主要な情報を集めた可視化されたページです。\n*   キーメトリックをリアルタイムで表示し、ビジネス上の成果と問題点を把握できます。\n*   カスタマイズが可能で、自分好みのレイアウトやデザインにすることができます。\n*   シェアリング機能があり、チーム内でダッシュボードを共有できます。'}

[Trace(trace_id=tr-420b0884b601fedfbdf5840b5d55e5eb), Trace(trace_id=tr-c407396a5d1b81f7602266f27ad644fc), Trace(trace_id=tr-5c4df1c6507149d0673accc14f8cc1df), Trace(trace_id=tr-edd2d0ecd8e7f2d3dfb0b2db4d1851cc), Trace(trace_id=tr-a90c4ce88493acbcce752b85dc78f0b7)]

In [0]:
import mlflow

from mlflow.genai.scorers import Guidelines

# 実験から最大10件のトレースをサンプリング
traces = mlflow.search_traces(max_results=10)

# スコアラーを使って評価を実行
with mlflow.start_run(run_name="eval_v1"):
    mlflow.genai.evaluate(
        data=traces,
        scorers=[
            Guidelines(
                name="mlflow_only",
                # ガイドラインはリクエストとレスポンスを参照可能
                guidelines="リクエストがMLflowに関連していない場合、応答は回答を拒否しなければなりません。",
            ),
        ],
    )

Evaluating:   0%|          | 0/5 [Elapsed: 00:00, Remaining: ?] 

<!DOCTYPE html>
 
 
 Evaluation output 
 
 
 
 
 
 
 
 
 View evaluation results.

[Trace(trace_id=tr-5c4df1c6507149d0673accc14f8cc1df), Trace(trace_id=tr-c407396a5d1b81f7602266f27ad644fc), Trace(trace_id=tr-edd2d0ecd8e7f2d3dfb0b2db4d1851cc), Trace(trace_id=tr-a90c4ce88493acbcce752b85dc78f0b7), Trace(trace_id=tr-420b0884b601fedfbdf5840b5d55e5eb)]

## ステップ 2: モニタリングを有効にする

それでは、モニタリングサービスを有効にしましょう。 有効にすると、モニタリング サービスは、評価されたトレースのコピーをMLflow エクスペリメントから、指定した スキーマのDelta Unity Catalogテーブルに同期します。

In [0]:
# これらのパッケージは mlflow[databricks] で自動的にインストールされます
from databricks.agents.monitoring import (
    create_external_monitor,
    AssessmentsSuiteConfig,
    GuidelinesJudge,
)

external_monitor = create_external_monitor(
    catalog_name="workspace",
    schema_name="monitor",
    assessments_config=AssessmentsSuiteConfig(
        sample=1.0,  # サンプリングレート
        assessments=[
            # ガイドラインはリクエストとレスポンスを参照可能
            GuidelinesJudge(
                guidelines={
                    # 任意の数のガイドラインを key-value で指定可能
                    "mlflow_only": ["リクエストがMLflowに関連していない場合、応答は回答を拒否しなければなりません。"]
                }
            ),
        ],
    ),
)

print(external_monitor)

---------------------------------------------------------------------------
HTTPError                                 Traceback (most recent call last)
File <command-3979169371715332>, line 8
      1 # これらのパッケージは mlflow[databricks] で自動的にインストールされます
      2 from databricks.agents.monitoring import (
      3     create_external_monitor,
      4     AssessmentsSuiteConfig,
      5     GuidelinesJudge,
      6 )
----> 8 external_monitor = create_external_monitor(
      9     catalog_name="workspace",
     10     schema_name="monitor",
     11     assessments_config=AssessmentsSuiteConfig(
     12         sample=1.0,  # サンプリングレート
     13         assessments=[
     14             # ガイドラインはリクエストとレスポンスを参照可能
     15             GuidelinesJudge(
     16                 guidelines={
     17                     # 任意の数のガイドラインを key-value で指定可能
     18                     "mlflow_only": ["リクエストがMLflowに関連していない場合、応答は回答を拒否しなければなりません。"]
     19                 }
     20             ),
     21         ],
 


# ステップ4.モニタリング結果の使用
モニタリング ジョブを初めて実行するには、~15 分から 30 分かかります。 最初の実行後、15 分ごとに実行されます。 本番運用のトラフィックが大量にある場合、ジョブの完了にさらに時間がかかる可能性があることに注意してください。

ジョブが実行されるたびに、次のことが行われます。

トレースのサンプルに対して各スコアラーを実行します

スコアラーごとにサンプリングレートが異なる場合、モニタリングジョブは、同じトレースをできるだけ多くスコアリングしようとします。たとえば、スコアラー A のサンプリング レートが 20%で、スコアラー B のサンプリング レートが 40% の場合、トレースの同じ 20% が A と B に使用されます。
スコアラーからの フィードバック を MLflow エクスペリメントの各トレースに添付します

すべてのトレース (サンプリングされたトレースだけでなく) のコピーを、手順 1 で構成された Delta テーブルに書き込みます。

モニタリング結果は、 MLflow エクスペリメントのTraceタブを使用して表示できます。または、生成された Delta テーブルで SQL または Spark を使用してトレースのクエリを実行することもできます。

In [0]:
# 追加のトレースを作成する
generate_feature_explanation("MLflow", "PromptRegistry")
generate_feature_explanation("python", "Pandas")
generate_feature_explanation("python", "numpy")
generate_feature_explanation("Databricks", "Model Serving")
generate_feature_explanation("MLflow", "LLM Evaluation")


{'explanation': 'MLflowのLLM Evaluation機能についての説明は以下のとおりです。\n\n*   LLM Evaluationは、MLflowのモデル評価機能の一部であり、大規模言語モデル(LLM)のパフォーマンスを評価するために使用されます。\n*   この機能により、開発者はLLMをさまざまなメトリックや基準で評価し、モデルの精度や有効性を把握できます。\n*   LLM Evaluationは、データセットやタスクに依存しない汎用的な評価フレームワークを提供し、開発者はLLMの強みと弱みを特定し、必要な最適化や改善を実施できます。\n*   これにより、LLMの信頼性と 安全性を高めることができ、より優れたAIソリューションの開発を支援します。'}

[Trace(trace_id=tr-3deb4ac1d13e534e488987fd7989c84e), Trace(trace_id=tr-f895b8c022dc3ce49b597e5a2fb2f7e4), Trace(trace_id=tr-a5089f208a45064ddc664226e4960dca), Trace(trace_id=tr-66068e1431194e1e0b8db3c629f2fc42), Trace(trace_id=tr-473769212534a264deb8da874bc447d0)]